In [28]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Preprocessing

We proprocess the test and the train at the same time in order to save time

In [29]:
data = pd.read_csv('data/avatars_echOvid_train_X.csv') 
data_test = pd.read_csv('data/avatars_echOvid_test_X.csv')

In [30]:
data.head()

,id,Nom du centre,age,Sexe,Lieu de provenance du patient,Echographiste,Tabagisme actif,BPCO,Asthme,Autre antecedent respiratoire,...,Zone anterieure droite haut,Zone anterieure droite bas,Zone anterieure gauche haut,Zone anterieure gauche bas,Zone posterieure droite haut,Zone posterieure droite bas,Zone posterieure gauche haut,Zone posterieure gauche bas,Oxygenotherapie,Outcome J28
0,1,Paris-APHP - Cochin,91,Masculin,Domicile,Forme pour l'epidemie,Non,Non,Non,Non,...,Stade 2,Stade 2,Stade 1,Stade 2,Stade 2,Stade 2,Stade 2,Stade 2,NaN,Death
1,109,Paris-APHP - Hotel Dieu,45,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Moderee,Back home
2,46,Melun - CH Marc Jacquet,76,Feminin,EHPAD,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 1,Stade 1,Stade 0,Stade 1,Stade 2,Stade 1,Stade 2,Assistance respiratoire,Hospitalization
3,247,Paris-APHP - Hotel Dieu,55,Feminin,Hopital,Experience d'echographie,Oui,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home
4,195,Paris-APHP - Hotel Dieu,42,Feminin,Domicile,Experience d'echographie,Non,Non,Non,Non,...,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,Stade 0,NaN,Back home


In [31]:
data.dtypes

id                                                                            int64
Nom du centre                                                                object
age                                                                           int64
Sexe                                                                         object
Lieu de provenance du patient                                                object
Echographiste                                                                object
Tabagisme actif                                                              object
BPCO                                                                         object
Asthme                                                                       object
Autre antecedent respiratoire                                                object
Hypertension arterielle                                                      object
Cardiopathie ischemique                                                     

## Features : Convert categorical variable into integer variables.

In [32]:
data.columns[data.isna().any()].tolist()
data_test.columns[data_test.isna().any()].tolist()

['Lieu de provenance du patient',
 'Echographiste',
 'Tabagisme actif',
 'Asthme',
 'Autre antecedent respiratoire',
 'Hypertension arterielle',
 'Cardiopathie ischemique',
 'Cardiopathie rythmique',
 'Diabete de type 2',
 'Cancer  hemopathie maligne',
 'Statut immunodeprime',
 "AINS au long cours (dans le cadre d'une pathologie suivie)",
 'AINS ponctuel recent (cadre des symptomatologies COVID-19 suspect avere)',
 'Tension arterielle systolique (mmHg)',
 'Tension arterielle diastolique (mmHg)',
 'Frequence cardiaque (puls. min)',
 'Frequence respiratoire (resp. min)',
 'Temperature (Celsius)',
 'Confusion',
 'Saturation O2',
 'Date de debut de la symptomatologie',
 'Oxygenotherapie']

In [33]:
data[data['Echographiste'].isna()]

,id,Nom du centre,age,Sexe,Lieu de provenance du patient,Echographiste,Tabagisme actif,BPCO,Asthme,Autre antecedent respiratoire,...,Zone anterieure droite haut,Zone anterieure droite bas,Zone anterieure gauche haut,Zone anterieure gauche bas,Zone posterieure droite haut,Zone posterieure droite bas,Zone posterieure gauche haut,Zone posterieure gauche bas,Oxygenotherapie,Outcome J28
11,159,Paris-APHP - Lariboisiere,68,Masculin,Domicile,NaN,Non,Non,Non,Non,...,Stade 2,Stade 2,Stade 3,Stade 2,Stade 1,Stade 3,Stade 2,Stade 3,Air ambiant,Hospitalization
252,272,Paris-APHP - Lariboisiere,83,Feminin,EHPAD,NaN,Non,Non,Non,Non,...,Stade 3,Stade 3,Stade 2,Stade 3,Stade 2,Stade 3,Stade 3,Stade 3,Moderee,Hospitalization


In [34]:
data['Sexe'].replace({"Masculin": 1, "Feminin": 0}, inplace=True)
data_test['Sexe'].replace({"Masculin": 1, "Feminin": 0}, inplace=True)

In [35]:
data['Lieu de provenance du patient'].replace({"Domicile": 0, "EHPAD": 1, "Hopital" : 2, 'Autre': 3, np.nan : 3}, inplace=True)
data_test['Lieu de provenance du patient'].replace({"Domicile": 0, "EHPAD": 1, "Hopital" : 2, 'Autre': 3, np.nan : 3}, inplace=True)

In [36]:
data['Echographiste'].replace({"Forme pour l'epidemie": 0, "Experience d'echographie": 1, "Expert" : 2, np.nan : 3}, inplace=True)
data_test['Echographiste'].replace({"Forme pour l'epidemie": 0, "Experience d'echographie": 1, "Expert" : 2, np.nan : 3}, inplace=True)

In [37]:
data['Nom du centre'].replace({"Paris-APHP - Cochin": 0, "Paris-APHP - Hotel Dieu": 1, "Melun - CH Marc Jacquet" : 2, 'Strasbourg - CHRU de Strasbourg - Hautepierre' : 3, 'Creteil - APHP - Henri Mondor' : 4, 'Montbrison - CH du Forez' : 5, 'Paris-APHP - Lariboisiere' : 6}, inplace=True)
data_test['Nom du centre'].replace({"Paris-APHP - Cochin": 0, "Paris-APHP - Hotel Dieu": 1, "Melun - CH Marc Jacquet" : 2, 'Strasbourg - CHRU de Strasbourg - Hautepierre' : 3, 'Creteil - APHP - Henri Mondor' : 4, 'Montbrison - CH du Forez' : 5, 'Paris-APHP - Lariboisiere' : 6}, inplace=True)

In [38]:
binary_columns = ['Tabagisme actif', 'BPCO', 'Asthme', 'Autre antecedent respiratoire', 'Hypertension arterielle', 'Cardiopathie ischemique', 'Cardiopathie rythmique', 'Diabete de type 1', 'Diabete de type 2', 'Diabetes', 'Cancer  hemopathie maligne', 'Demence', 'Statut immunodeprime', "AINS au long cours (dans le cadre d'une pathologie suivie)", 'AINS ponctuel recent (cadre des symptomatologies COVID-19 suspect avere)', 'Confusion']

In [39]:
for column in binary_columns:
    data[column].replace({"Oui": 1, "Non": 0, np.nan : 2}, inplace=True)
    data_test[column].replace({"Oui": 1, "Non": 0, np.nan : 2}, inplace=True)

In [40]:
zone_header = ['Zone anterieure droite haut',
               'Zone anterieure droite bas',
               'Zone anterieure gauche haut',
               'Zone anterieure gauche bas',
               'Zone posterieure droite haut',
               'Zone posterieure droite bas',
               'Zone posterieure gauche haut',
               'Zone posterieure gauche bas']

In [41]:
for zone in zone_header:
    data[zone].replace({"Stade 0": 0, "Stade 1": 1, "Stade 2" : 2, "Stade 3" : 3}, inplace=True)
    data_test[zone].replace({"Stade 0": 0, "Stade 1": 1, "Stade 2" : 2, "Stade 3" : 3}, inplace=True)

In [42]:
data['Oxygenotherapie'].replace({"Moderee": 0, "Air ambiant": 1, 'Assistance respiratoire' : 2, np.nan : 3}, inplace=True)
data_test['Oxygenotherapie'].replace({"Moderee": 0, "Air ambiant": 1, 'Assistance respiratoire' : 2, np.nan : 3}, inplace=True)

## Labels : Convert categorical variable into integer variables.

In [43]:
data['Outcome J28'].replace({"Death": 0, "Back home": 1, 'Hospitalization' : 2, 'Intensive care unit' : 3}, inplace=True)

## Nan values to solve

In [44]:
data.columns[data.isna().any()].tolist()

['Tension arterielle systolique (mmHg)',
 'Tension arterielle diastolique (mmHg)',
 'Frequence cardiaque (puls. min)',
 'Frequence respiratoire (resp. min)',
 'Temperature (Celsius)',
 'Saturation O2',
 'Date de debut de la symptomatologie']

In [45]:
data['Tabagisme actif'].unique()

array([0, 1, 2])

In [46]:
data

,id,Nom du centre,age,Sexe,Lieu de provenance du patient,Echographiste,Tabagisme actif,BPCO,Asthme,Autre antecedent respiratoire,...,Zone anterieure droite haut,Zone anterieure droite bas,Zone anterieure gauche haut,Zone anterieure gauche bas,Zone posterieure droite haut,Zone posterieure droite bas,Zone posterieure gauche haut,Zone posterieure gauche bas,Oxygenotherapie,Outcome J28
0,1,0,91,1,0,0,0,0,0,0,...,2,2,1,2,2,2,2,2,3,0
1,109,1,45,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,46,2,76,0,1,1,0,0,0,0,...,0,1,1,0,1,2,1,2,2,2
3,247,1,55,0,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,3,1
4,195,1,42,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,218,0,34,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,3,1
258,199,2,58,0,0,1,0,0,0,0,...,1,1,0,0,1,1,0,1,2,2
259,117,0,45,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
260,201,0,46,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1


In [47]:
imputer = KNNImputer(n_neighbors=5)
data[['Tension arterielle systolique (mmHg)', 'Tension arterielle diastolique (mmHg)', 'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)', 'Temperature (Celsius)', 'Saturation O2']] = imputer.fit_transform(data[['Tension arterielle systolique (mmHg)', 'Tension arterielle diastolique (mmHg)', 'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)', 'Temperature (Celsius)', 'Saturation O2']])
data_test[['Tension arterielle systolique (mmHg)', 'Tension arterielle diastolique (mmHg)', 'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)', 'Temperature (Celsius)', 'Saturation O2']] = imputer.fit_transform(data_test[['Tension arterielle systolique (mmHg)', 'Tension arterielle diastolique (mmHg)', 'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)', 'Temperature (Celsius)', 'Saturation O2']])

In [48]:
data['Tension arterielle systolique (mmHg)'] = data['Tension arterielle systolique (mmHg)'].round(decimals = 0)
data['Tension arterielle diastolique (mmHg)'] = data['Tension arterielle diastolique (mmHg)'].round(decimals = 0)
data['Frequence cardiaque (puls. min)'] = data['Frequence cardiaque (puls. min)'].round(decimals = 0)
data['Frequence respiratoire (resp. min)'] = data['Frequence respiratoire (resp. min)'].round(decimals = 0)
data['Saturation O2'] = data['Saturation O2'].round(decimals = 0)
data['Temperature (Celsius)'] = data['Temperature (Celsius)'].round(decimals = 1)

In [49]:
data_test['Tension arterielle systolique (mmHg)'] = data_test['Tension arterielle systolique (mmHg)'].round(decimals = 0)
data_test['Tension arterielle diastolique (mmHg)'] = data_test['Tension arterielle diastolique (mmHg)'].round(decimals = 0)
data_test['Frequence cardiaque (puls. min)'] = data_test['Frequence cardiaque (puls. min)'].round(decimals = 0)
data_test['Frequence respiratoire (resp. min)'] = data_test['Frequence respiratoire (resp. min)'].round(decimals = 0)
data_test['Saturation O2'] = data_test['Saturation O2'].round(decimals = 0)
data_test['Temperature (Celsius)'] = data_test['Temperature (Celsius)'].round(decimals = 1)

# Remove useless column

In [50]:
data = data.drop(columns=['Date de debut de la symptomatologie'])
data_test = data_test.drop(columns=['Date de debut de la symptomatologie'])

# Add a column for each label

In [51]:
#Uncomment code bellow for MLP
#data['Intensive care unit & death2'] = np.where((data['Outcome J28'] == 0) | (data['Outcome J28'] == 3), 1, 0)
data['Death'] = np.where(data['Outcome J28'] == 0, 1, 0)
data['Back home'] = np.where(data['Outcome J28'] == 1, 1, 0)
data['Hospitalization'] = np.where(data['Outcome J28'] == 2, 1, 0)
data['Intensive care unit'] = np.where(data['Outcome J28'] == 3, 1, 0)
data['Intensive care unit & death'] = np.where((data['Outcome J28'] == 1) | (data['Outcome J28'] == 2), 1, data['Outcome J28'])
data['Intensive care unit & death'] = np.where(data['Outcome J28'] == 3, 2, data['Outcome J28'])

In [52]:
data.columns

Index(['id', 'Nom du centre', 'age', 'Sexe', 'Lieu de provenance du patient',
       'Echographiste', 'Tabagisme actif', 'BPCO', 'Asthme',
       'Autre antecedent respiratoire', 'Hypertension arterielle',
       'Cardiopathie ischemique', 'Cardiopathie rythmique',
       'Diabete de type 1', 'Diabete de type 2', 'Diabetes',
       'Cancer  hemopathie maligne', 'Demence', 'Statut immunodeprime',
       'AINS au long cours (dans le cadre d'une pathologie suivie)',
       'AINS ponctuel recent (cadre des symptomatologies COVID-19 suspect avere)',
       'Tension arterielle systolique (mmHg)',
       'Tension arterielle diastolique (mmHg)',
       'Frequence cardiaque (puls. min)', 'Frequence respiratoire (resp. min)',
       'Temperature (Celsius)', 'Confusion', 'Saturation O2',
       'Zone anterieure droite haut', 'Zone anterieure droite bas',
       'Zone anterieure gauche haut', 'Zone anterieure gauche bas',
       'Zone posterieure droite haut', 'Zone posterieure droite bas',
      

# Save files

In [53]:
!rm data/clean_data.csv
data.to_csv("data/clean_data.csv", index=False)
!rm data/clean_test_data.csv
data_test.to_csv("data/clean_test_data.csv", index=False)

In [54]:
!rm data/clean_data.csv
data.to_csv("data/clean_data.csv", index=False)